In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
#train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/train.csv')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = df.sample(64000)
df = df.drop(train.index)
print(df)
test = df

       Unnamed: 0                                   Paraphrased Text  Author
0               0  Vaccination not required for people with mild ...       3
8               0  I am so happy that you have joined my group, y...       3
11              0  More than 2 million vaccine doses are being co...       3
12              0  The social media community is fighting to surv...       3
14              0  A new study shows that the Delta L2 spreader i...       3
...           ...                                                ...     ...
79977           0  A pink's Jung Eun Ji and Hayoung were diagnose...       0
79979           0  I've had a lot of questions about my health in...       3
79980           0  The culture of covid-19 will be made sure that...       1
79992           0  We wish the family and friends of Nikhil a hap...       3
79997           0  In the first 20 days of 2020, Israel has treat...       1

[16000 rows x 3 columns]


In [4]:
test.rename(columns = {'Paraphrased Text':'Text'}, inplace = True)
train.rename(columns = {'Paraphrased Text':'Text'}, inplace = True)

In [5]:
test

,Unnamed: 0,Text,Author
0,0,Vaccination not required for people with mild ...,3
8,0,"I am so happy that you have joined my group, y...",3
11,0,More than 2 million vaccine doses are being co...,3
12,0,The social media community is fighting to surv...,3
14,0,A new study shows that the Delta L2 spreader i...,3
...,...,...,...
79977,0,A pink's Jung Eun Ji and Hayoung were diagnose...,0
79979,0,I've had a lot of questions about my health in...,3
79980,0,The culture of covid-19 will be made sure that...,1
79992,0,We wish the family and friends of Nikhil a hap...,3


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))
#
# Any results you write to the current directory are saved as output.

In [7]:
#train = pd.read_csv('../input/train.tsv' , delimiter= '\t')
#test = pd.read_csv('../input/test.tsv', delimiter= '\t')

In [8]:
train = train.drop(['Unnamed: 0'], axis=1)
test = test.drop(['Unnamed: 0'], axis=1)

The sentiment labels are:
0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive

In [9]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack, vstack

import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
plt.figure(figsize=(8,10))
sentiment_counts = train.Author.value_counts()
#sns.barplot(sentiment_counts.index , sentiment_counts.values)
plt.show()

<Figure size 800x1000 with 0 Axes>

Looks like neutral sentiments are more. This should be because of the phrases present in the dataset

In [11]:
train_y = train['Author'].values

My initial approach is to do a word and character tfidf and Countvectorizer on the text data.  The result of tfidf and BOW will be a sparse matri and hence I will go ahead with TruncatedSVD for dimensionality reduction. Since the term frequency is considered in TfIdf, I am ensembling the predictions of Bag of Words approach to the SVD as my final feature set

In [ ]:
train['Text'].values.tolist()

In [ ]:
train['Text'].values.astype('U').tolist()

In [25]:
tfv = TfidfVectorizer(ngram_range=(1,3), use_idf= True , analyzer= 'word')
tfv.fit(train['Text'].values.astype('U').tolist() + test['Text'].values.astype('U').tolist())
train_tfidf = tfv.transform(train['Text'].values.astype('U'))
test_tfidf = tfv.transform(test['Text'].values.astype('U'))

In [26]:
cv_object = CountVectorizer(ngram_range=(1,3), analyzer= 'word')
cv_object.fit(train['Text'].values.astype('U').tolist() + test['Text'].values.astype('U').tolist())
train_cv = cv_object.transform(train['Text'].values.astype('U'))
test_cv = cv_object.transform(test['Text'].values.astype('U'))

In [27]:
tf_char = TfidfVectorizer(ngram_range=(1,6), analyzer= 'char' , max_features=20000)
tf_char.fit(train['Text'].values.astype('U').tolist() + test['Text'].values.astype('U').tolist())
train_tfidf_char = tf_char.transform(train['Text'].values.astype('U'))
test_tfidf_char = tf_char.transform(test['Text'].values.astype('U'))

In [28]:
bow_char = CountVectorizer(ngram_range=(1,6), analyzer= 'char' , max_features=20000)
bow_char.fit(train['Text'].values.astype('U').tolist() + test['Text'].values.astype('U').tolist())
train_cv_char = bow_char.transform(train['Text'].values.astype('U'))
test_cv_char = bow_char.transform(test['Text'].values.astype('U'))

In [29]:
def SVD(train , test, keyword, n_components = 25):
    svd_obj = TruncatedSVD(n_components=n_components, algorithm='arpack')
    svd_obj.fit(vstack([train, test]))
    columns = ['svd_'+ keyword + '_' + str(i) for i in range(n_components)]
    train_df = pd.DataFrame(data = svd_obj.transform(train) , columns = columns)
    test_df = pd.DataFrame(data = svd_obj.transform(test) , columns = columns)
    return train_df, test_df

In [30]:
print(train_tfidf.shape)
print(test_tfidf.shape)

(64000, 807889)
(16000, 807889)


In [31]:
final_train , final_test = SVD(train_tfidf, test_tfidf , 'word_tfidf')

In [32]:
tf_char_train,  tf_char_test = SVD(train_tfidf_char, test_tfidf_char, 'char_tfidf')

In [33]:
final_train = pd.concat([final_train, tf_char_train], axis=1)
final_test = pd.concat([final_test, tf_char_test], axis= 1)

In [34]:
del tf_char_train, tf_char_test, train_tfidf, test_tfidf

In [35]:
def model_multinomial(train_X, train_y, test_X, test_y, test_X2):
    model = MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

In [36]:
def calculate_cv_score(model, train_x , train_y, test_x, num_splits = 3, loss = log_loss, is_dataframe = False):
    ''' model needs to return validation prediction , test prediction and model itself after fitting'''
    cv_scores = []
    pred_train = np.zeros([train_x.shape[0] ,4])
    pred_test_final = 0
    kfold = KFold(n_splits= num_splits, random_state= 2018 , shuffle= True)
    
    for dev_index , val_index in kfold.split(train_x):
        
        if is_dataframe:
            dev_X, val_X = train_x.loc[dev_index], train_x.loc[val_index]
        else:            
            dev_X, val_X = train_x[dev_index], train_x[val_index]

        
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        pred_val , pred_test , fit_model = model(dev_X, dev_y, val_X , val_y , test_x)
        pred_test_final = pred_test_final + pred_test
        loss_score = loss(val_y , pred_val)
        pred_train[val_index,:] = pred_val
        cv_scores.append(loss_score)
    avg_cv_score = np.mean(cv_scores)
    pred_test_final = pred_test_final / num_splits
    print(pred_train.shape)
    return avg_cv_score, pred_test_final, pred_train 
        
        

In [37]:
train.shape

(64000, 2)

In [38]:
test_cv.shape

(16000, 807889)

In [39]:
calculate_cv_score(model_multinomial, train_cv, train_y, test_cv)

(64000, 4)


(2.609868969428043,
 array([[2.50041674e-12, 2.57426427e-14, 2.34437726e-09, 9.99999998e-01],
        [1.96060110e-02, 1.36351994e-02, 9.06743326e-01, 6.00154639e-02],
        [1.27036882e-11, 1.63446975e-09, 6.42148645e-01, 3.57851354e-01],
        ...,
        [2.80414037e-04, 1.55593578e-01, 3.69900350e-04, 8.43756107e-01],
        [2.78207313e-04, 9.16821359e-07, 9.99496613e-01, 2.24262739e-04],
        [6.15685605e-02, 2.17094560e-02, 2.57771774e-02, 8.90944806e-01]]),
 array([[9.06584565e-07, 8.74982209e-05, 5.01654413e-01, 4.98257182e-01],
        [5.22674272e-07, 2.39669106e-03, 1.16698023e-02, 9.85932984e-01],
        [4.58147951e-01, 4.75483710e-04, 8.23218024e-05, 5.41294243e-01],
        ...,
        [1.37539752e-04, 1.21008931e-03, 1.20806605e-02, 9.86571710e-01],
        [8.93055887e-03, 9.03836577e-04, 2.18028747e-01, 7.72136858e-01],
        [7.00000799e-01, 2.98950432e-02, 5.04779882e-03, 2.65056359e-01]]))

In [40]:
cvscore_bow_words, bow_predictions_words, pred_train = calculate_cv_score(model_multinomial, train_cv, train_y, test_cv)

(64000, 4)


In [41]:
cvscore_bow_char, bow_predictions_char, pred_train_char = calculate_cv_score(model_multinomial, train_cv_char, train_y, test_cv_char)

(64000, 4)


In [42]:
final_train["mnb_bow_word_0"] = pred_train[:,0]
final_train["mnb_bow_word_1"] = pred_train[:,1]
final_train["mnb_bow_word_2"] = pred_train[:,2]
final_train["mnb_bow_word_3"] = pred_train[:,3]
#final_train["mnb_bow_word_4"] = pred_train[:,4]
final_test["mnb_bow_word_0"] = bow_predictions_words[:,0]
final_test["mnb_bow_word_1"] = bow_predictions_words[:,1]
final_test["mnb_bow_word_2"] = bow_predictions_words[:,2]
final_test["mnb_bow_word_3"] = bow_predictions_words[:,3]
#final_test["mnb_bow_word_4"] = bow_predictions_words[:,4]

In [43]:
final_train

,svd_word_tfidf_0,svd_word_tfidf_1,svd_word_tfidf_2,svd_word_tfidf_3,svd_word_tfidf_4,svd_word_tfidf_5,svd_word_tfidf_6,svd_word_tfidf_7,svd_word_tfidf_8,svd_word_tfidf_9,...,svd_char_tfidf_19,svd_char_tfidf_20,svd_char_tfidf_21,svd_char_tfidf_22,svd_char_tfidf_23,svd_char_tfidf_24,mnb_bow_word_0,mnb_bow_word_1,mnb_bow_word_2,mnb_bow_word_3
0,0.067422,0.011743,-0.006668,0.009683,0.005628,-0.017061,-0.027311,0.004150,-0.010674,-0.019623,...,-0.029460,-0.051498,-0.026648,-0.092797,0.040046,0.052679,9.065846e-07,8.749822e-05,0.501654,0.498257
1,0.047555,0.031599,-0.015726,0.005668,-0.015738,0.006663,0.007993,-0.005230,-0.001943,-0.016959,...,0.115110,0.050940,0.079589,-0.057670,-0.012092,-0.009603,5.226743e-07,2.396691e-03,0.011670,0.985933
2,0.075388,-0.028648,-0.026184,-0.015689,-0.000551,0.009115,0.011137,-0.003491,-0.017309,-0.000204,...,-0.016971,0.022107,0.005854,-0.029639,0.003739,0.041461,4.581480e-01,4.754837e-04,0.000082,0.541294
3,0.037936,0.001747,0.002444,0.008073,-0.014442,-0.010005,-0.003030,-0.019562,-0.001881,0.007161,...,-0.012730,0.035986,0.008447,0.038966,-0.067347,-0.059835,3.525910e-01,6.236907e-01,0.001466,0.022253
4,0.070021,0.024707,-0.007069,0.004581,-0.016195,-0.008965,-0.005897,-0.018069,0.009149,0.001764,...,-0.035506,-0.006572,-0.005093,0.052248,0.062602,-0.051503,3.060141e-08,5.599752e-07,0.999975,0.000024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63995,0.068645,-0.021423,-0.046144,0.076670,-0.081399,0.018499,0.018444,-0.024970,0.048254,-0.010109,...,-0.106506,-0.108959,0.068931,0.004020,0.005279,-0.009411,4.901296e-08,5.265505e-07,0.296472,0.703528
63996,0.091409,-0.053116,-0.016141,-0.035784,-0.007768,-0.017428,-0.027052,-0.030958,0.011500,0.008540,...,-0.043105,-0.103363,0.007469,-0.011476,0.019844,0.078217,7.923048e-06,2.927458e-07,0.775358,0.224633
63997,0.054866,-0.066887,-0.029346,-0.021680,0.026712,-0.004291,-0.015664,-0.006120,0.015817,0.039140,...,0.006135,0.032039,0.045360,-0.013982,-0.054597,-0.023745,1.375398e-04,1.210089e-03,0.012081,0.986572
63998,0.088527,-0.001785,0.006697,0.010147,0.015518,-0.017533,-0.033494,-0.002969,0.000799,-0.007446,...,0.021198,0.044574,-0.030726,-0.056340,-0.003193,-0.000665,8.930559e-03,9.038366e-04,0.218029,0.772137


In [44]:
final_train["mnb_bow_char_0"] = pred_train_char[:,0]
final_train["mnb_bow_char_1"] = pred_train_char[:,1]
final_train["mnb_bow_char_2"] = pred_train_char[:,2]
final_train["mnb_bow_char_3"] = pred_train_char[:,3]
#final_train["mnb_bow_char_4"] = pred_train_char[:,4]

final_test["mnb_bow_char_0"] = bow_predictions_char[:,0]
final_test["mnb_bow_char_1"] = bow_predictions_char[:,1]
final_test["mnb_bow_char_2"] = bow_predictions_char[:,2]
final_test["mnb_bow_char_3"] = bow_predictions_char[:,3]
#final_test["mnb_bow_char_4"] = bow_predictions_char[:,4]

In [45]:
print(final_test.shape)
print(final_train.shape)
print(train_y.shape)

(16000, 58)
(64000, 58)
(64000,)


In [46]:
def run_lr(train_X, train_y, val_X, val_y, test_X):
    lr = LogisticRegression(C=0.01)
    lr.fit(train_X, train_y)
    pred_val = lr.predict_proba(val_X)
    pred_test = lr.predict_proba(test_X)
    return pred_val, pred_test, lr

In [47]:
final_cv, final_pred, _ = calculate_cv_score(run_lr, final_train, train_y, final_test, is_dataframe=True)

(64000, 4)


In [48]:
sub1 = np.argmax(final_pred, axis=1)

In [49]:
np.bincount(sub1)

array([3884, 3927, 4306, 3883])

In [50]:
test['Sentiment'] = sub1

In [51]:
test

,Text,Author,Sentiment
0,Vaccination not required for people with mild ...,3,3
8,"I am so happy that you have joined my group, y...",3,2
11,More than 2 million vaccine doses are being co...,3,2
12,The social media community is fighting to surv...,3,3
14,A new study shows that the Delta L2 spreader i...,3,3
...,...,...,...
79977,A pink's Jung Eun Ji and Hayoung were diagnose...,0,0
79979,I've had a lot of questions about my health in...,3,1
79980,The culture of covid-19 will be made sure that...,1,1
79992,We wish the family and friends of Nikhil a hap...,3,2


In [68]:
test['Author']

0        3
8        3
11       3
12       3
14       3
        ..
79977    0
79979    3
79980    1
79992    3
79997    1
Name: Author, Length: 16000, dtype: int64

In [70]:
test['Author'].values[]

3

In [71]:
accuracy = 0
for i in range (0,len(test)):
  if test['Author'].values[i] == test['Sentiment'].values[i]:
    accuracy += 1

accuracy = accuracy/len(test)

print(accuracy)


0.589125


In [ ]:
#test[['PhraseId' , 'Sentiment']].to_csv('sub_1.csv' , index=False)